# Montaje

In [ ]:
#from google.colab import drive, files
import sys

#drive.mount('/content/drive', force_remount=True)
# '/content/drive/My Drive/Universidad/Proyectos/NubesKeras'
#sys.path.append('/content/drive/My Drive/Proyectos/NubesKeras')

import tensorflow as tf
import seaborn as sns
import numpy as np
import pandas as pd

from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

from math import ceil

config = tf.ConfigProto(log_device_placement=False, allow_soft_placement=True)    
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.90
sess = tf.Session(config = config)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
!ls


# Carga de datos

In [ ]:
from Model import *
from Utils import *

file_dir = "data"
model_dir = "results"

data = load_data(file_dir)

data_generator = make_data_generator(data['train'])

# Aprendizaje pre-builts

#### VGG19

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
model_name = 'vgg19'
fit_model(data['train'], data['valid'], data_generator,
          make_prebuilt(VGG19, .1),
          batch_size=128,
          model_name=model_name,
          model_dir=model_dir)

save_results(model_dir, model_name, data['label_encoder'],
             data_generator, test_data=data['test'])


In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
model_name = 'vgg19_alt'
fit_model(data['train'], data['valid'], data_generator,
          make_prebuilt_extended(VGG19, .1), 
          model_name=model_name,
          model_dir=model_dir)

save_results(model_dir, model_name, data['label_encoder'],
             data_generator, test_data=data['test'])


#### Inception V3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
model_name = 'inceptionv3'
fit_model(data['train'], data['valid'], data_generator,
                        make_prebuilt(InceptionV3,.1), model_name=model_name,
                        model_dir=model_dir)

save_results(model_dir, model_name, data['label_encoder'],
             data_generator, test_data=data['test'])

#### Inception ResNet V2

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
model_name = 'inceptionresnetv2'
fit_model(data['train'], data['valid'], data_generator,
                        make_prebuilt(InceptionResNetV2,.25), model_name=model_name,
                        model_dir=model_dir)
save_results(model_dir, model_name, data['label_encoder'],
             data_generator, test_data=data['test'])

#### NASNetLarge

In [ ]:
from tensorflow.keras.applications.nasnet import NASNetLarge
model_name = 'nasnetlarge'
fit_model(data['train'], data['valid'], data_generator,
                        make_prebuilt(NASNetLarge, .5, wgh=None), model_name=model_name,
                        model_dir=model_dir, batch_size=32)
save_results(model_dir, model_name, data['label_encoder'],
             data_generator, test_data=data['test'])

# Random Forest Ensembles

##### NN into RF

In [ ]:
model_name = 'inceptionv3'
model = load_model('%s/%s' % (model_dir, '%s_model.h5' % model_name))
model.summary()

# Training rf
print('Train')
x_train = data['features'][0]
y_train = data['label_encoder'].inverse_transform(data['train'][-1])

print('tr - Train classifier')
classifier = RandomForestClassifier(250, random_state=1,
                                    max_features=.35)

classifier.fit(x_train, y_train)

print('Test')
img_test, ceil_test, y_test= data['test']
x_test = data['features'][-1]
standard_img_test = data_generator.standardize(img_test)

print('te - Get predictions from trained model')
nn_pred = model.predict([standard_img_test, ceil_test])
rf_pred = classifier.predict_proba(x_test)
avg_pred = (nn_pred + rf_pred)/2
max_pred = np.maximum(nn_pred, rf_pred)

rf_pred = data['label_encoder'].inverse_transform(rf_pred)
nn_pred = data['label_encoder'].inverse_transform(nn_pred)
avg_pred = data['label_encoder'].inverse_transform(avg_pred)
max_pred = data['label_encoder'].inverse_transform(max_pred)

decoded_observations = data['label_encoder'].inverse_transform(y_test)
pred_obs = pd.DataFrame(data={'avg_pred': avg_pred,
                              'max_pred': max_pred,
                              'obs': decoded_observations,
                              'rf_pred': rf_pred,
                              'nn_pred': nn_pred})


print('---{}---'.format('ensemble_avg'))
print(classification_report(pred_obs['obs'], pred_obs['avg_pred'], digits=4))
print('---{}---'.format('ensemble_max'))
print(classification_report(pred_obs['obs'], pred_obs['max_pred'], digits=4))
print('---{}---'.format('random_forest'))
print(classification_report(pred_obs['obs'], pred_obs['rf_pred'], digits=4))
print('---{}---'.format(model_name))
print(classification_report(pred_obs['obs'], pred_obs['nn_pred'], digits=4))

# Evaluar el Modelo
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
matrix = confusion_matrix(pred_obs['obs'], pred_obs['avg_pred'])
#names = pred_obs['obs'].unique()
sns.heatmap(matrix, annot=True, cbar=False)

ensemble_name='inceptionresnetv2_plus_RandomForest'
pred_obs.to_csv('%s/%s' % (file_dir, '%s_preds.csv' % ensemble_name))



#### NN and RF

In [ ]:
model_name = 'inceptionresnetv2'
model = load_model('%s/%s' % (model_dir, '%s_model.h5' % model_name))
  
rf_model = Model(inputs=model.input, outputs=model.layers[-5].output)
rf_model.summary()

# Training rf
print('Train')
img_train, ceil_train, y_train = data['train']
print('tr - Get predictions from trained model')
_, att = rf_model.output.shape
num = img_train.shape[0]
batch_size = 128

#train_prediction_set = rf_model.predict_generator(data_generator.flow(x=(img_train,ceil_train),
#                                                                        batch_size=batch_size),
#                                                                        steps= ceil(num/batch_size),
#                                                                        verbose=1)
y_train_dec = list(data['label_encoder'].inverse_transform(y_train))

reductor = PCA(.95, svd_solver='full')
classifier = RandomForestClassifier(100, random_state=1,
                                    max_features=.35, n_jobs=8)

print('tr - Dimensionality reduction')
x = reductor.fit_transform(train_prediction_set)
print(x.shape)

print('tr - Train classifier')
classifier.fit(x, y_train_dec)
decoded_predictions = classifier.predict(x)
pred_obs = pd.DataFrame(data={'pred': decoded_predictions, 'obs': y_train_dec})

print(classification_report(pred_obs['obs'], pred_obs['pred']))

# Testing rf and saving results
print('Test')
img_test, ceil_test, y_test= data['test']
print('te - Standarize test data')
standard_img_test = data_generator.standardize(img_test)
print('te - Get predictions from trained model')
test_predictions = rf_model.predict([standard_img_test, ceil_test])
test_predictions = reductor.transform(test_predictions)
print('te - Final predictions')
decoded_predictions = classifier.predict(test_predictions)
decoded_observations = data['label_encoder'].inverse_transform(y_test)
pred_obs = pd.DataFrame(data={'pred': decoded_predictions, 'obs': decoded_observations})

# Evaluar el Modelo
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')

matrix = confusion_matrix(pred_obs['obs'], pred_obs['pred'])
names = pred_obs['obs'].unique()

sns.heatmap(matrix, annot=True, cbar=False, xticklabels=names, yticklabels=names)
print(classification_report(pred_obs['obs'], pred_obs['pred']))

# Almacenar las predicciones del modelo entrenado
# pred_obs.to_csv('%s/%s' % (file_dir, '%s_preds.csv' % model_name))

# Aprendizaje Cropnet

#### CropNet v1

In [ ]:
model_name = 'cropnetv1'
fit_model(data['train'], data['valid'], data_generator,
                        make_cropnetv1, model_name=model_name,
                        model_dir=model_dir, lr=1e-3)
save_results(model_dir, model_name, data['label_encoder'],
             data_generator, test_data=data['test'])

#### CropNet v2

In [ ]:
model_name = 'cropnetv2'
fit_model(data['train'], data['valid'], data_generator,
                        make_cropnetv2, model_name=model_name,
                        model_dir=model_dir, n_outputs=6)
save_results(model_dir, model_name, data['label_encoder'],
             data_generator, test_data=data['test'], n_outputs=6)

#### Cropnet v3

In [ ]:

model_name = 'cropnetv3'
fit_model(data['train'], data['valid'], data_generator,
                        make_cropnetv3, model_name=model_name,
                        model_dir=model_dir, n_outputs=6, batch_size=32)
save_results(model_dir, model_name, data['label_encoder'],
             data_generator, test_data=data['test'], n_outputs=6)

#### RandomCropnet v1

In [ ]:
model_name = 'rcropnetv1'
fit_model(data['train'], data['valid'], data_generator,
                        make_rcropnetv1(16), model_name=model_name,
                        model_dir=model_dir, n_outputs=18, batch_size=32)
save_results(model_dir, model_name, data['label_encoder'],
             data_generator, test_data=data['test'], n_outputs=18)

#### RandomCropnet v2

In [ ]:
model_name = 'rcropnetv2'
fit_model(data['train'], data['valid'], data_generator,
                        make_rcropnetv2(32, 128), model_name=model_name,
                        model_dir=model_dir, n_outputs=32+2, batch_size=32)
save_results(model_dir, model_name, data['label_encoder'],
             data_generator, test_data=data['test'], n_outputs=18)